In [12]:
using SQLite
import SQLite.Stmt
import SQLite.DBInterface.execute
using Distributions

In [13]:
db1 = SQLite.DB("output/before/sweep_db_gathered.sqlite")
db2 = SQLite.DB("output/after/sweep_db_gathered.sqlite")

SQLite.DB("output/after/sweep_db_gathered.sqlite")

In [14]:
import StatsAPI.pvalue
import StatsBase.mean
import StatsBase.mad
import HypothesisTests.ApproximateTwoSampleKSTest

# Functions to compare using K-S tests

"""
    Do K-S tests using SQLite queries that take one parameter, run_id.

    `db1` and `db2` are assumed to have identical parameter combinations labeled the same way,
    via the `combo_id` column in the `runs` table.

    The query should return a single value for the provided run_id.

    If values from db2 need to be extracted with a different query, provide a value for `q2`.
"""
function compare(db1, db2, q1; q2 = q1)
    combo_ids = get_combo_ids(db1)
    @assert all(combo_ids .== get_combo_ids(db2))
    [(combo_id, compare(db1, db2, combo_id, q1; q2 = q2)) for combo_id in combo_ids]
end

function get_combo_ids(db)
    [combo_id for (combo_id,) in execute(db, "SELECT DISTINCT combo_id FROM runs ORDER BY combo_id")]
end

function get_run_ids(db, combo_id)
    [run_id for (run_id,) in execute(db, "SELECT run_id FROM runs WHERE combo_id = ?", [combo_id])]
end

function compare(db1, db2, combo_id, q1; q2 = q1)
    v1 = query_values_for_combo_id(db1, combo_id, q1)
    mean1 = mean(v1)
    mad1 = mad(v1; center = mean1)

    v2 = query_values_for_combo_id(db2, combo_id, q2)
    mean2 = mean(v2)
    mad2 = mad(v2; center = mean2)

    (ApproximateTwoSampleKSTest(v1, v2), (mean1, mad1), (mean2, mad2))
end

function query_values_for_combo_id(db, combo_id, q)
    stmt = Stmt(db, q)
    run_ids = get_run_ids(db, combo_id)

    [query_value_for_run_id(stmt, run_id) for run_id in run_ids]
end

function query_value_for_run_id(stmt, run_id)
    for (value,) in execute(stmt, (run_id,))
        return value
    end
end

query_value_for_run_id (generic function with 1 method)

In [15]:
for (combo_id, (test, (mean1, mad1), (mean2, mad2))) in compare(
    db1, db2, "SELECT value FROM run_meta WHERE key = \"elapsed_time\" AND run_id = ?"
)
    println("Testing elapsed time for parameter combination $(combo_id)...")
    println("    (mean1 $(mean1), mad1 $(mad1))")
    println("    (mean2 $(mean2), mad2 $(mad2))")
    println("    p-value: $(pvalue(test))")
    if pvalue(test) < 0.01
        println("    DIFFERENT distributions w/ p < 0.01")
    else
        println("    undetectable difference between distributions w/ p < 0.01")
    end
    println("    fractional difference: $((mean1 - mean2) / mean1)")
end

Testing elapsed time for parameter combination 1...
    (mean1 215.06039999999996, mad1 4.030602391229385)
    (mean2 266.1723333333333, mad2 5.390247465746886)
    p-value: 6.118046410036549e-7
    DIFFERENT distributions w/ p < 0.01
    fractional difference: -0.23766315571501478


## Mean # active infected hosts from 90 to 100 years

In [16]:
for (combo_id, (test, (mean1, mad1), (mean2, mad2))) in compare(
    db1, db2, "SELECT AVG(n_infected_active) FROM summary WHERE run_id = ? AND (time >= 9 * 360) AND (time <= 100 * 360)"
)
    println("Testing mean infected for parameter combination $(combo_id)...")
    println("    (mean1 $(mean1), mad1 $(mad1))")
    println("    (mean2 $(mean2), mad2 $(mad2))")
    println("    p-value: $(pvalue(test))")
    if pvalue(test) < 0.01
        println("    DIFFERENT distributions w/ p < 0.01")
    else
        println("    undetectable difference between distributions w/ p < 0.01")
    end
    println("    fractional difference: $((mean1 - mean2) / mean1)")
end

Testing mean infected for parameter combination 1...
    (mean1 311.75384615384615, mad1 15.829629840659786)
    (mean2 266.46153846153845, mad2 13.79960526455216)
    p-value: 4.22910789740631e-6
    DIFFERENT distributions w/ p < 0.01
    fractional difference: 0.1452822739834189


## Mean # circulating genes from 90 to 100 years

In [17]:
for (combo_id, (test, (mean1, mad1), (mean2, mad2))) in compare(
    db1, db2, "SELECT AVG(n_circulating_genes_blood) FROM gene_strain_counts WHERE run_id = ? AND (time >= 9 * 360) AND (time <= 10 * 360)"
)
    println("Testing mean infected for parameter combination $(combo_id)...")
    println("    (mean1 $(mean1), mad1 $(mad1))")
    println("    (mean2 $(mean2), mad2 $(mad2))")
    println("    p-value: $(pvalue(test))")
    if pvalue(test) < 0.01
        println("    DIFFERENT distributions w/ p < 0.01")
    else
        println("    undetectable difference between distributions w/ p < 0.01")
    end
    println("    fractional difference: $((mean1 - mean2) / mean1)")
end

Testing mean infected for parameter combination 1...
    (mean1 1199.8666666666666, mad1 0.1976802958009042)
    (mean2 1199.4, mad2 0.8895613311032262)
    p-value: 0.18130044993812283
    undetectable difference between distributions w/ p < 0.01
    fractional difference: 0.0003889321035668988
